In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [ ]:
# chọn biến mục tiêu
target = 'loan_status'
model_features = list(set(df.columns).difference({target}))

X = df[model_features]
y = df[target]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=0.1)

### GridSearchCV

##### GridSearch for Gini

In [ ]:
def GridSearch_for_Gini(x_set, y_set, steps):
    lr = LogisticRegression(solver='newton-cg', random_state = 0)
    
    #Setting the range for class weights
    weights = np.linspace(0.0,0.99,20)
    
    #Creating a dictionary grid for grid search
    param_grid = {'class_weight': [{0:1-x, 1:x} for x in weights]}
    
    #Fitting grid search to the train data with 5 folds
    gridsearch = GridSearchCV(estimator= lr, 
                              param_grid= param_grid,
                              cv=StratifiedKFold(), 
                              n_jobs=-1, 
                              scoring='roc_auc', 
                              verbose=2).fit(x_set, y_set)
    

    # Tạo bảng kết quả gridsearch gồm weights và scores
    weigh_data = pd.DataFrame({'score': gridsearch.cv_results_['mean_test_score'], 'weight': weights})
    #Ploting the results
    sns.set_style('whitegrid')
    plt.figure(figsize=(9,6))
    plot = sns.lineplot(x = weigh_data['weight'], y = weigh_data['score'])  # set x-axis là weight, y-axis là score
    plt.xlabel('Weight for class 1')
    plt.ylabel('AUC score')
    plt.xticks([round(i/10,1) for i in range(0,11,1)])
    plt.title('Scoring for different class weights', fontsize=24)
    return plot

##### Finding optimal threshold for f1

In [ ]:
# Do parameter threshold không có sẵn nên em chạy thủ công bằng vòng for :)))
threshold = np.linspace(0, 0.99, 20)       #tạo range cần chạy cho threshold, trong đó 20 là số bước nhảy
for threshold in threshold:
    # xây mô hình
    lr = LogisticRegression(solver='newton-cg', random_state = 0)
    lr.fit(x_train, y_train)
    
    # Predicting on the test data
    pred_test = lr.predict_proba(x_test)
    
    pred_test = pd.DataFrame(pred_test,
                         columns = ['0', '1'])     # Tạo DataFrame của kết quả xác suất ước lượng
    # convert sang giá trị 1 hoặc 0 dựa trên threshold
    pred_test_binary = pd.DataFrame(np.where(pred_test['1'] >= threshold, 1, 0), columns = ['1'])
    #f1 score
    f1.append(f1_score(y_test, pred_test_binary))
    
    # tạo bảng kết quả gridsearch
    f1 = pd.DataFrame(f1, columns = ['score'])       
    f1['threshold'] = threshold
    # Plot the results
    sns.set_style('whitegrid')
    sns.lineplot(x = f1['threshold'], y = f1['score'])    #set x_axis = threshold, y-axis = f1_score
    plt.xlabel('Weight for class 1')
    plt.ylabel('f1_score')
    plt.xticks([round(i/10,1) for i in range(0,11,1)])
    plt.title('f1_score for different threshold', fontsize=24)

In [ ]:
def conf_matrix(y_test, pred_test_binary):    
    
    # Creating a confusion matrix
    con_mat = confusion_matrix(y_test, pred_test_binary)
    con_mat = pd.DataFrame(con_mat, range(2), range(2))
   
    #Ploting the confusion matrix
    plt.figure(figsize=(6,6))
    sns.set(font_scale=1.5) 
    sns.heatmap(con_mat, annot=True, annot_kws={"size": 16}, fmt='g', cmap='Blues', cbar=False)

### Forecast

In [ ]:
def forecast(w1, threshold):         # w1: weight of class 1
    lr = LogisticRegression(solver='newton-cg', class_weight={0: 1-w1, 1: w1}, random_state = 0)     
    lr.fit(x_train, y_train)            # xây mô hình
    
    # Predicting on the test data
    pred_test = lr.predict_proba(x_test)
    pred_test = pd.DataFrame(pred_test,
                         columns = ['0', '1'])      # Tạo DataFrame của kết quả xác suất ước lượng
    
    # convert sang giá trị 1 hoặc 0 dựa trên threshold
    pred_test_binary = pd.DataFrame(np.where(pred_test['1'] >= threshold, 1, 0), columns = ['1']) 
    #Calculating and printing the f1 score 
    print('The f1 score for the testing data:', f1_score(y_test, pred_test_binary))
    print('The Gini coefficient for the testing data:', 2*roc_auc_score(y_test, pred_test['1']) - 1)

    #Ploting the confusion matrix
    conf_matrix(y_test, pred_test_binary)

### Finding P-value

#### Methodology

\begin{equation}
P_{value} = P(Z > |z|)
\end{equation}
where,\
Z ~ Normal(0,1)\
z = Estimated Coef/standard error(s.e)


##### Calculate standard error
\begin{equation}
s.e = \sqrt{(X^T\hat{W}X)^{-1}}
\end{equation}

X là ma trận gồm cột số 1 và tập X_train\
$X^T$ là transpose của ma trận X\
W là ma trận 0 và có đường chéo là $\hat{y}(1-\hat{y})$

In [ ]:
#Data Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[model_features])

In [ ]:
X = scaler.transform(df[model_features])
y = df[target]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=0.3)

In [ ]:
# x_set là tập train
# pred_set là cột giá trị ước lượng dựa trên tập train, KHÔNG DỰA TRÊN TẬP TEST
# model: tên model
def summary_model(x_set, pred_set, model):                                    
    #tạo ma trận X
    X_design = np.hstack([np.ones((len(x_set), 1)), x_set])    
    
    # Tạo ma trận 0 có đường chéo là y(1-y)
    W = np.diagflat(pred_set[:, 1]*(1-pred_set[:, 1]))

    # Covariance matrix = X^T * W * X
    covLogit = np.linalg.pinv(X_design.T @ V @ X_design)
    
    # Standard errors
    standard_error = np.sqrt(np.diag(covLogit))
    
    #Create summary table
    summary_table = pd.DataFrame(model.coef_.T, columns = ['Coef'])             # Tạo bảng các estimated coef
    summary_table.set_index(pd.DataFrame(x_set).columns, inplace = True)        # set index là tên columns
    summary_table['s.e'] = standard_error[1:,]                                  # Thêm cột standard error
    summary_table['z'] = summary_table['Coef']/summary_table['s.e']             # Thêm cột z nhỏ, z = beta/s.e
    summary_table['p_value'] = stats.norm.cdf(-abs(summary_table['z'])) * 2     # Tính p-value từ z nhỏ
    return summary_table